In [ ]:
###
# Prerekvizity: stažení dat z webu
import requests
import os
if not os.path.exists("B2BTUR01.xls"):
    url = "http://portal.chmi.cz/files/portal/docs/meteo/ok/denni_data/files/B2BTUR01.xls"
    r = requests.get(url)
    open("B2BTUR01.xls", 'wb').write(r.content)

In [ ]:
###
# instalace balíku pro Excel
# možná přes příkaz:
#%pip install xlrd
import pandas as pd
import numpy as np

## Načtení dat
Pomůže nám funkce `read_excel`, která bere za parametry
* soubor
* `sheet_name` - jedna hodnota nebo seznam integerových pořadí listů či jejich názvů
* `header` - na kterém řádku je hlavička

Vrací jeden DataFrame nebo sérii DataFramů (pokud je více listů)

In [ ]:
df_list = pd.read_excel("B2BTUR01.xls", sheet_name=[1, 2, 3], header=3)
df_avg, df_max, df_min =  df_list.values() ###

In [ ]:
# Podívejme se na data, co jsme načetli
df_avg

In [ ]:
# A na datové typy
df_avg.info()

In [ ]:
# Nyní musíme napsat funkci, která nám upraví (vyčistí dataframe)
# - do "stacked" formátu, název sloupce bude podle parametru value_name
# - odstraní diakritiku ze sloupce (funkce rename)
# - přetypuje den ze stringu do do integeru (a odstraní tečku)

def df_update(df : pd.DataFrame, value_name : str) -> pd.DataFrame: ###
    df = df.melt(value_vars=df.columns[2:].values, var_name="den", value_name=value_name, id_vars=["rok", "měsíc"])
    df.rename(columns = {"měsíc": "mesic"}, inplace=True)
    df["den"] = df["den"].str.replace(".", "").astype("int")
    return df ###

df_update(df_avg, "temp_avg") ###

In [ ]:
###
# Provedeme pro všechny dataframy
df_avg_s = df_update(df_avg, "temp_avg")
df_min_s = df_update(df_min, "temp_min")
df_max_s = df_update(df_max, "temp_max")

In [ ]:
# nyí všechny DataFrame musíme spojit dohromady
# - varianta jedna pomocí merge
pd.merge(pd.merge(df_avg_s, df_min_s, on=["rok", "mesic", "den"]), df_max_s, on=["rok", "mesic", "den"])

In [ ]:
# Nastavíme správně indexy
df_avg_i = df_avg_s.set_index(["rok", "mesic", "den"])
df_min_i = df_min_s.set_index(["rok", "mesic", "den"])
df_max_i = df_max_s.set_index(["rok", "mesic", "den"])

In [ ]:
# složíme data (ale správná osa)
df_all = pd.concat([df_avg_i, df_min_i, df_max_i], axis=1)
df_all

# Hrátky s daty
Data máme připravená v pěkném formátu, můžeme se nyní dotazovat!

## Kdy u nás byla nejnižší teplota?
(7. ledna 1985)

In [ ]:
# kdy u nas byla nejnizsi teplota
df_all["temp_min"].min()
df_all["temp_min"].argmin()
df_all.iloc[df_all["temp_min"].argmin()]

## Jak bylo 30. října v různých rocích?

In [ ]:
# jak bylo 30. rijna v ruzýnch rocich?
tmp = df_all.loc[(slice(None), 10, 30)]
tmp

In [ ]:
# Zkusme vizualizovat do sloupcového grafu
import matplotlib.pyplot as plt ###
plt.Figure() ###
plt.bar(x=tmp.index.values, height=tmp["temp_avg"].to_numpy())
plt.show() ###

## Analýza jednotlivých měsíců
Budeme hledat nějaké měsíce v historii, kdy nastaly různé jevy.

In [ ]:
# Vytvořme dataframe, kde pro každý měsíc a rok bude minimální a maximální teplota
###
df_mesic = df_all.groupby(["rok", "mesic"]).agg({"temp_min": "min", "temp_max": "max"})
# Pro jednodušší práci resetujme indexy
df_mesic.reset_index(inplace=True) ###
df_mesic ###

### V kterých měsících mrzlo?

In [ ]:
# V kterých měsících mrzlo?
df_mesic[df_mesic["temp_min"] < 0]

In [ ]:
# mrzlo nekdy v květnu?
df_mesic[(df_mesic["temp_min"]<0) & (df_mesic["mesic"] == 5)]

In [ ]:
# kolikrát v jednotlivých měsících mrzlo?
df_mesic[df_mesic["temp_min"] < 0].groupby("mesic")["rok"].count()

In [ ]:
# Ztratily se nám ale některé měsíce.
dfc = df_mesic[["mesic", "temp_min"]].copy()  # Vytvoříme kopii ###
# Místo toho nahradíme nenulové 
dfc.loc[dfc["temp_min"] < 0, "freeze"] = True
dfc
dfc.groupby("mesic").agg( {"freeze" : "sum"} ) #.plot.bar()

## Jaké byly největší teplotní skoky?

In [ ]:
# jake byly nejvetsi teplotni skoky?
df_range = df_mesic.copy()
df_range["range"] = df_mesic["temp_max"] - df_mesic["temp_min"]
df_range

In [ ]:
# Vypiš prvních 10 měsíců, kdy byl největší teplotní skok
df_range.sort_values("range", ascending=False).head(10)

In [ ]:
df_range.nlargest(columns="range", n=10)

## Můžeme se vrátit zpět k formátu z CHMI?

In [ ]:
df_all.dropna().unstack(level="den")["temp_avg"]